In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
# trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
# devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
# testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


12152
2867
3005


In [4]:
print(testSentences[3])

[['Die', 'O'], ['sechs', 'O'], ['Abgeordneten', 'O'], ['der', 'O'], ['Grünen', 'B-ORG'], [',', 'O'], ['die', 'O'], ['seit', 'O'], ['Bildung', 'O'], ['einer', 'O'], ['Großen', 'O'], ['Koalition', 'O'], ['von', 'O'], ['Christ-', 'O'], ['und', 'O'], ['Sozialdemokraten', 'O'], ['die', 'O'], ['Opposition', 'O'], ['darstellen', 'O'], [',', 'O'], ['hatten', 'O'], ['sich', 'O'], ['beim', 'O'], ['Vorsteher', 'O'], ['Rainer', 'B-PER'], ['Bergert', 'I-PER'], ['für', 'O'], ['ihr', 'O'], ['Fehlen', 'O'], ['entschuldigen', 'O'], ['lassen', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

9
205


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-LOC': 1, 'B-PER': 5, 'PADDING_TOKEN': 0, 'O': 7, 'B-LOC': 6, 'I-ORG': 4, 'I-PER': 3, 'B-MISC': 2, 'B-ORG': 8, 'I-MISC': 9}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'contains_digit': 7, 'mainly_numeric': 6, 'PADDING_TOKEN': 0, 'other': 5, 'allUpper': 3, 'allLower': 2, 'initialUpper': 4, 'numeric': 1}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'á': 102, '4': 5, 'j': 6, 'r': 7, '(': 59, 'u': 60, 'w': 8, 'q': 62, 'N': 9, '-': 10, '=': 53, 'é': 65, 'I': 66, 'E': 11, 'ñ': 12, 'ì': 33, '5': 68, 'z': 69, 'A': 54, '&': 70, 'd': 101, 'S': 71, 'W': 73, 'g': 14, '7': 61, 'Q': 15, '<S>': 1, 'J': 16, 'v': 17, 'UNKNOWN': 103, '2': 83, 'p': 19, '0': 75, '6': 76, '.': 63, 'ç': 96, 'm': 21, '"': 22, ':': 77, "'": 78, '?': 23, 'F': 64, 'D': 18, 'ß': 81, 'G': 82, '+': 74, '<W>': 3, 'L': 84, ')': 20, 'ó': 45, 'è': 28, 'a': 29, 'M': 87, '§': 97, ',': 30, 'V': 25, '/': 99, 'x': 88, 'y': 32, 'Ä': 89, 'c': 34, 'Ö': 90, '1': 35, 'B': 91, 'o': 36, '</S>': 2, '*': 37, '</W>': 4, 'n': 38, 'ô': 39, 'C': 40, 'P': 41, 'X': 43, 'Z': 27, 'ê': 92, '9': 44, 'ü': 93, 'R': 94, 'PADDING_TOKEN': 0, 'O': 95, 'k': 13, 'b': 46, '%': 86, 'f': 98, 'h': 67, 'Y': 47, 'H': 85, ';': 48, '8': 49, 'l': 50, 'ò': 51, 'ö': 100, '!': 26, 'ä': 52, 't': 24, 'i': 55, 'í': 56, '3': 79, 'Ü': 42, 'e': 31, 's': 80, 'U': 57, 'T': 58, 'K': 72}


In [14]:
print(trainSentences[0]) 

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
5


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

10
10


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'tmp_3cnn_bi-lstm_conll.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_3cnn()

In [ ]:
#import importlib
#importlib.reload(utils)
# print(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
#print(models.idx2Label)
#print(true_labels[0:10])
#print(pred_labels[0:10])
label_pred = []    
for sentence in pred_labels:
    label_pred.append([models.idx2Label[element] for element in sentence])
print(label_pred)

# Experiments

In [19]:
f = open('results_3cnn_conll.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_3cnn_conll_best.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_3cnn()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 3328        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_10 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time

ResourceExhaustedError: OOM when allocating tensor with shape[391960,32,1,53] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: time_distributed_13/convolution/Conv2D = Conv2D[T=DT_FLOAT, _class=["loc:@train...kpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_13/convolution/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, time_distributed_13/convolution/ExpandDims_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training_1/Nadam/gradients/time_distributed_13/convolution/Squeeze_grad/Shape/_763 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edg

Epoch 1/10
23/24 [===========================>..] - ETA: 5s - loss: -0.0114 - acc: 0.9994 
New maximum F1 score: 0.8516497791634191 (before: 0.8496152341202557) Saving to tmp_generator_NER_3cnn_conll_best.0.h5
24/24 [==============================] - 237s 10s/step - loss: -0.0114 - acc: 0.9994 - val_loss: -0.0073 - val_acc: 0.9981
Epoch 2/10
23/24 [===========================>..] - ETA: 3s - loss: -0.0116 - acc: 0.9994
New maximum F1 score: 0.8525615956198672 (before: 0.8516497791634191) Saving to tmp_generator_NER_3cnn_conll_best.0.h5
24/24 [==============================] - 199s 8s/step - loss: -0.0116 - acc: 0.9994 - val_loss: -0.0074 - val_acc: 0.9981
Epoch 3/10
23/24 [===========================>..] - ETA: 4s - loss: -0.0119 - acc: 0.9995
New maximum F1 score: 0.8556607443911294 (before: 0.8525615956198672) Saving to tmp_generator_NER_3cnn_conll_best.0.h5
24/24 [==============================] - 204s 8s/step - loss: -0.0119 - acc: 0.9995 - val_loss: -0.0075 - val_acc: 0.9981
Epoch

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close